Example to show detect hand written text using a usb camera. Printing rectangles on recognized text.
Press space to capture and space and esc to quit.

In [ ]:
import numpy as np
import cv2
import time
import requests
from PIL import Image
import matplotlib.patches as patches
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
from matplotlib.patches import Polygon

# PROXIES IF YOU ARE RUNNING THE JUPYTER NOTEBOOK FROM YOUR LAPTOP
proxy = "http://www:8080"
proxys = "https://www:8080"
proxyDict = { "http"  : proxy, "https"  : proxys }

%matplotlib inline 

def recognize_text_azure(image_path):
    subscription_key = ''
    assert subscription_key
    vision_base_url = "https://PYTHONCOMPUTERVISION.cognitiveservices.azure.com/vision/v1.0/"
    ocr_url = vision_base_url + "recognizeText"
    image_data = open(image_path, "rb").read()
    headers    = {'Ocp-Apim-Subscription-Key': subscription_key, 
              "Content-Type": "application/octet-stream" }

    params   = {'mode' : 'Handwritten'}
    #data     = {'url': image_url}
    response = requests.post(ocr_url, headers=headers, params=params, data=image_data)#, proxies=proxyDict)
    response.raise_for_status()

    operation_url = response.headers["Operation-Location"]

    analysis = {}
    while not "recognitionResult" in analysis:
        response_final = requests.get(response.headers["Operation-Location"], headers=headers)#, proxies=proxyDict)
        analysis       = response_final.json()
        time.sleep(3)

    polygons = [(line["boundingBox"], line["text"]) for line in analysis["recognitionResult"]["lines"]]

    return polygons
    
    
#image = cv2.imread('friends..JPG')
def detect_and_show_text():

    cam = cv2.VideoCapture(0,cv2.CAP_DSHOW)

    cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cv2.namedWindow("Text Detector")
    img_counter = 0

    while True:
        start = time.time()
        ret, frame = cam.read()
        #CALLING FUNCTION
        
        cv2.imshow("text detector", frame)
        
        if not ret:
            break
        k = cv2.waitKey(1)

        if k%256 == 27:
            # ESC pressed
            print("Escape hit, closing...")
            break
        elif k%256 == 32:
            # SPACE pressed
            img_name = "temp/opencv_frame_{}.png".format(img_counter)
            cv2.imwrite(img_name, frame)
            print("{} written!".format(img_name))
            polygons = recognize_text_azure(img_name)
            
            image = cv2.imread(img_name)


            for polygon in polygons:
                vertices = [(polygon[0][i], polygon[0][i+1]) for i in range(0,len(polygon[0]),2)]
                text     = polygon[1]
                print(text,vertices,' ',' ',vertices[0],vertices[1])
                cv2.rectangle(image, vertices[0] , vertices[2],(0, 0, 255), 2)
                cv2.putText(image, text, vertices[0], cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
            cv2.imshow('recognized text', image)
            while ( cv2.waitKey(0) != 32 ):
                time.sleep(1)
            img_couqnter += 1
            

    cam.release()

    cv2.destroyAllWindows()
    

  


In [ ]:
detect_and_show_text()